In [1]:
with open("/mnt/d/forCoding_code/_init_tools/x__params.py", "r") as f:
    txt = f.read()
exec(txt)

with open("/mnt/d/forCoding_code/_init_tools/report_macro_0516.py", "r") as f:
    txt = f.read()
exec(txt)

with open("/mnt/d/forCoding_code/_init_tools/x__tool_here1.py", "r") as f:
    exec(f.read())
with open("/mnt/d/forCoding_code/_init_tools/x__tool_here2.py", "r") as f:
    exec(f.read())

wasted_dir = "./wasted"
if not os.path.exists(wasted_dir):
    os.makedirs(wasted_dir)

pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

storage dir: /mnt/d/forCoding_data/Tianchi_Ecommerce6/plan_1
code dir: /mnt/d/forCoding_code/Tianchi_Ecommerce6/plan_1


In [15]:
## 写一个代码，生成一个classification任务的样本我看看。
## 你似乎只打出了一个例子，能多打几个出来吗？
import os
import torch
from transformers import BertTokenizerFast as BertTokenizer

from config import Config
from utils_loc import load_train_data, ClassificationDataset
from train import prepare_classification_samples


def show_one(idx: int, samples, dataset, tokenizer, max_tokens: int):
    raw_sample = samples[idx]
    item = dataset[idx]

    input_ids = item["input_ids"]
    token_type_ids = item["token_type_ids"]
    attention_mask = item["attention_mask"]
    cat_labels = item.get("cat_labels")
    pol_labels = item.get("pol_labels")

    print("=" * 80)
    print(f"样本索引 idx = {idx}")
    print("---- 原始 classification 样本（prepare_classification_samples 输出）----")
    print(raw_sample)

    print("\n---- 张量形态（送进 ClassificationModel 的）----")
    print(f"input_ids.shape      : {tuple(input_ids.shape)}")
    print(f"token_type_ids.shape : {tuple(token_type_ids.shape)}")
    print(f"attention_mask.shape : {tuple(attention_mask.shape)}")

    if cat_labels is not None and pol_labels is not None:
        cat_idx = int(cat_labels.item())
        pol_idx = int(pol_labels.item())
        cat = Config.IDX2CAT[cat_idx]
        pol = Config.IDX2POL[pol_idx]
        print(f"\n分类标签 cat_labels : {cat_idx} -> {cat}")
        print(f"情感标签 pol_labels : {pol_idx} -> {pol}")

    tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist())
    print(f"\n---- 前 {max_tokens} 个 token（含特殊符号）----")
    for i, tok in enumerate(tokens[:max_tokens]):
        tt = int(token_type_ids[i].item())
        am = int(attention_mask[i].item())
        print(f"{i:03d}  token={tok:8s}  token_type={tt}  attn={am}")


def main():
    tokenizer = BertTokenizer.from_pretrained(Config.BERT_PATH)

    reviews_df, labels_df = load_train_data()
    samples = prepare_classification_samples(reviews_df, labels_df)
    print(f"总样本数: {len(samples)}")

    dataset = ClassificationDataset(reviews_df, samples, tokenizer, Config.MAX_LEN)

    # 控制要看的起始索引和数量
    start_idx = int(os.environ.get("START_IDX", "0"))
    num_samples = int(os.environ.get("NUM_SAMPLES", "5"))
    max_tokens = int(os.environ.get("MAX_TOKENS", "40"))

    end_idx = min(start_idx + num_samples, len(samples))
    print(f"将展示样本索引 [{start_idx}, {end_idx - 1}]")

    for idx in range(start_idx, end_idx):
        show_one(idx, samples, dataset, tokenizer, max_tokens)


if __name__ == "__main__":
    main()

总样本数: 11393
将展示样本索引 [0, 4]
样本索引 idx = 0
---- 原始 classification 样本（prepare_classification_samples 输出）----
{'id': 1, 'aspect': None, 'opinion': '很好', 'category': '整体', 'polarity': '正面'}

---- 张量形态（送进 ClassificationModel 的）----
input_ids.shape      : (512,)
token_type_ids.shape : (512,)
attention_mask.shape : (512,)

分类标签 cat_labels : 8 -> 整体
情感标签 pol_labels : 0 -> 正面

---- 前 40 个 token（含特殊符号）----
000  token=[CLS]     token_type=0  attn=1
001  token=很         token_type=0  attn=1
002  token=好         token_type=0  attn=1
003  token=，         token_type=0  attn=1
004  token=超         token_type=0  attn=1
005  token=值         token_type=0  attn=1
006  token=，         token_type=0  attn=1
007  token=很         token_type=0  attn=1
008  token=好         token_type=0  attn=1
009  token=用         token_type=0  attn=1
010  token=[SEP]     token_type=0  attn=1
011  token=隐         token_type=1  attn=1
012  token=式         token_type=1  attn=1
013  token=[SEP]     token_type=1  attn=1
014  token=很  

In [18]:
## 两类负样本，都搞点样例给我看看？

import os
from utils_loc import load_train_data
from train import prepare_classification_samples


def fmt_asp(a):
    return a if a is not None else "_(隐式)"


def truncate(s, n):
    s = "" if s is None else str(s)
    return s if len(s) <= n else (s[:n] + "...")


def main():
    num_rids = int(os.environ.get("NUM_RIDS", "5"))
    max_pos = int(os.environ.get("MAX_POS", "8"))
    max_neg1 = int(os.environ.get("MAX_NEG1", "6"))
    max_neg2 = int(os.environ.get("MAX_NEG2", "6"))
    max_text = int(os.environ.get("MAX_TEXT", "120"))
    only_rid = os.environ.get("RID", "").strip()

    reviews_df, labels_df = load_train_data()
    review_map = reviews_df.set_index("id")["Reviews"].astype(str).to_dict()

    samples = prepare_classification_samples(reviews_df, labels_df)

    by_rid = {}
    for s in samples:
        rid = int(s["id"])
        by_rid.setdefault(rid, []).append(s)

    if only_rid:
        rid_list = [int(only_rid)]
    else:
        candidates = []
        for rid, arr in by_rid.items():
            has_pos = any(x.get("category") != "None" for x in arr)
            has_neg = any(x.get("category") == "None" for x in arr)
            if has_pos and has_neg:
                candidates.append(rid)
        rid_list = sorted(candidates)[:num_rids]

    print(f"总 samples: {len(samples)}")
    print(f"将展示 rid 数量: {len(rid_list)}")

    for rid in rid_list:
        arr = by_rid.get(rid, [])
        pos = [x for x in arr if x.get("category") != "None"]
        neg1 = [x for x in arr if x.get("category") == "None" and x.get("aspect") is not None]
        neg2 = [x for x in arr if x.get("category") == "None" and x.get("aspect") is None]

        print("\n" + "=" * 100)
        print(f"RID = {rid}")
        print(f"Review = {truncate(review_map.get(rid, ''), max_text)}")

        print("\n[正样本] category != 'None'（真实四元组/正例）")
        if not pos:
            print("  (无)")
        else:
            for i, s in enumerate(pos[:max_pos]):
                print(
                    f"  ({i}) aspect={fmt_asp(s['aspect'])}  opinion={s['opinion']}  "
                    f"category={s['category']}  polarity={s['polarity']}"
                )

        print("\n[负样本类型1] 显式A + 显式O 的错配（aspect!=None 且 category=='None'）")
        if not neg1:
            print("  (无)")
        else:
            for i, s in enumerate(neg1[:max_neg1]):
                print(
                    f"  ({i}) aspect={fmt_asp(s['aspect'])}  opinion={s['opinion']}  "
                    f"category={s['category']}  polarity={s['polarity']}"
                )

        print("\n[负样本类型2] 隐式A(None) + 显式O（aspect==None 且 category=='None'）")
        if not neg2:
            print("  (无)")
        else:
            for i, s in enumerate(neg2[:max_neg2]):
                print(
                    f"  ({i}) aspect={fmt_asp(s['aspect'])}  opinion={s['opinion']}  "
                    f"category={s['category']}  polarity={s['polarity']}"
                )


if __name__ == "__main__":
    main()

总 samples: 11393
将展示 rid 数量: 5

RID = 2
Review = 很好，遮暇功能差一些，总体还不错

[正样本] category != 'None'（真实四元组/正例）
  (0) aspect=_(隐式)  opinion=很好  category=整体  polarity=正面
  (1) aspect=遮暇功能  opinion=差一些  category=功效  polarity=负面
  (2) aspect=_(隐式)  opinion=还不错  category=整体  polarity=正面

[负样本类型1] 显式A + 显式O 的错配（aspect!=None 且 category=='None'）
  (0) aspect=遮暇功能  opinion=很好  category=None  polarity=中性
  (1) aspect=遮暇功能  opinion=还不错  category=None  polarity=中性

[负样本类型2] 隐式A(None) + 显式O（aspect==None 且 category=='None'）
  (0) aspect=_(隐式)  opinion=差一些  category=None  polarity=中性

RID = 3
Review = 包装太随便了，连个包装盒都没有，第一感觉很不好

[正样本] category != 'None'（真实四元组/正例）
  (0) aspect=包装  opinion=太随便了  category=包装  polarity=负面
  (1) aspect=包装盒  opinion=没有  category=包装  polarity=负面
  (2) aspect=_(隐式)  opinion=很不好  category=整体  polarity=负面

[负样本类型1] 显式A + 显式O 的错配（aspect!=None 且 category=='None'）
  (0) aspect=包装盒  opinion=很不好  category=None  polarity=中性
  (1) aspect=包装盒  opinion=太随便了  category=None  polarity=中性
  (2) aspect

In [26]:
!head /mnt/d/forCoding_data/Tianchi_Ecommerce6/TRAIN/Train_labels.csv

id,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end,Categories,Polarities
1,_, , ,很好,0,2,整体,正面
1,_, , ,超值,3,5,价格,正面
1,_, , ,很好用,6,9,整体,正面
2,_, , ,很好,0,2,整体,正面
2,遮暇功能,3,7,差一些,7,10,功效,负面
2,_, , ,还不错,13,16,整体,正面
3,包装,0,2,太随便了,2,6,包装,负面
3,包装盒,9,12,没有,13,15,包装,负面
3,_, , ,很不好,20,23,整体,负面


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
!head /mnt/d/forCoding_data/Tianchi_Ecommerce6/TRAIN/Train_reviews.csv

id,Reviews
1,很好，超值，很好用
2,很好，遮暇功能差一些，总体还不错
3,包装太随便了，连个包装盒都没有，第一感觉很不好
4,宝贝收到了，产品非常的不好，简直就是个垃圾，我都扔了。
5,活动价很是划算，买一送一共60片才花八十五块，天天用都不心疼啊
6,使用一段时间才来评价，淡淡的香味，喜欢！
7,之前买过。用了还不错。继续回购
8,很润相对好推开。总体还可以。
9,收到了，当天晚上就用了很不错哦，


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import os
import sys
import torch
from transformers import BertTokenizerFast as BertTokenizer
from config import Config
from utils_loc import load_train_data, ExtractionDataset

def show_sample(dataset, idx=0, limit=30):
    item, text, rid, offset_mapping = dataset[idx]
    input_ids = item["input_ids"]
    attention_mask = item["attention_mask"]
    token_type_ids = item["token_type_ids"]
    labels = item.get("labels")

    print(f"id: {rid}")
    print(f"text: {text}")
    print(f"input_ids.shape: {tuple(input_ids.shape)}")
    print(f"attention_mask.sum: {int(attention_mask.sum().item())}")
    if labels is not None:
        print(f"labels.shape: {tuple(labels.shape)}")

    tokenizer = BertTokenizer.from_pretrained(Config.BERT_PATH)
    tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist())

    cnt = 0
    for i in range(len(tokens)):
        if attention_mask[i].item() != 1:
            continue
        start, end = offset_mapping[i].tolist()
        if start == 0 and end == 0:
            continue
        tok = tokens[i]
        tag = None
        if labels is not None:
            tag_idx = int(labels[i].item())
            tag = Config.IDX2TAG.get(tag_idx, str(tag_idx))
        line = f"{i:03d}  token={tok}  span=({start},{end})  text='{text[start:end]}'"
        if tag is not None:
            line += f"  tag={tag}"
        print(line)
        cnt += 1
        if cnt >= limit:
            break

# def main():
idx = int(os.environ.get("IDX", "0"))
limit = int(os.environ.get("LIMIT", "30"))

print("loading tokenizer and dataset ...")
tokenizer = BertTokenizer.from_pretrained(Config.BERT_PATH)
reviews, labels = load_train_data()
dataset = ExtractionDataset(reviews, labels, tokenizer, Config.MAX_LEN)
print(f"dataset size: {len(dataset)}")

show_sample(dataset, idx=idx, limit=limit)

/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading tokenizer and dataset ...
dataset size: 3229
id: 1
text: 很好，超值，很好用
input_ids.shape: (512,)
attention_mask.sum: 11
labels.shape: (512,)
001  token=很  span=(0,1)  text='很'  tag=B-OP
002  token=好  span=(1,2)  text='好'  tag=I-OP
003  token=，  span=(2,3)  text='，'  tag=O
004  token=超  span=(3,4)  text='超'  tag=B-OP
005  token=值  span=(4,5)  text='值'  tag=I-OP
006  token=，  span=(5,6)  text='，'  tag=O
007  token=很  span=(6,7)  text='很'  tag=B-OP
008  token=好  span=(7,8)  text='好'  tag=I-OP
009  token=用  span=(8,9)  text='用'  tag=I-OP


In [4]:
reviews

,id,Reviews
0,1,很好，超值，很好用
1,2,很好，遮暇功能差一些，总体还不错
2,3,包装太随便了，连个包装盒都没有，第一感觉很不好
3,4,宝贝收到了，产品非常的不好，简直就是个垃圾，我都扔了。
4,5,活动价很是划算，买一送一共60片才花八十五块，天天用都不心疼啊
...,...,...
3224,3225,***，大品牌值得信赖，皮肤亮白！服务周到！
3225,3226,之前也有买过，这是并不是第一次购买，应该是之前一样吧，价格挺优惠的
3226,3227,物流很快，包装不错，第一次使用这个牌子，期待&hellip
3227,3228,感觉还可以，唯一的遗憾就是面膜放了香精，希望***官方能把香精去掉就完美了


In [9]:
labels[labels.id==14]

,id,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end,Categories,Polarities
27,14,_,,,很好的,0,3,整体,正面
28,14,_,,,不错,28,30,整体,正面


In [10]:
labels[labels.AspectTerms!="_"]

,id,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end,Categories,Polarities
4,2,遮暇功能,3,7,差一些,7,10,功效,负面
6,3,包装,0,2,太随便了,2,6,包装,负面
7,3,包装盒,9,12,没有,13,15,包装,负面
12,6,香味,14,16,淡淡的,11,14,气味,正面
26,13,物流,15,17,快,18,19,物流,正面
...,...,...,...,...,...,...,...,...,...
6624,3227,物流,0,2,很快,2,4,物流,正面
6625,3227,包装,5,7,不错,7,9,包装,正面
6626,3228,香精,17,19,_,,,成分,负面
6629,3229,补水,15,17,棒棒的,17,20,功效,正面


In [11]:
labels.Categories.value_counts()

Categories
整体      2822
使用体验    1042
功效       726
价格       696
物流       517
气味       225
包装       195
真伪       161
服务        86
其他        65
成分        61
尺寸        24
新鲜度       13
Name: count, dtype: int64

In [ ]:
({'input_ids': tensor([ 101, 2523, 1962, 8024, 6631,  966, 8024, 2523, 1962, 4500,  102,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
、]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]),
  'labels': tensor([0, 3, 4, 0, 3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
])},
 '很好，超值，很好用',
 1,
 tensor([[0, 0],
         [0, 1],
         [1, 2],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]))

In [13]:
dataset[1]

({'input_ids': tensor([ 101, 2523, 1962, 8024, 6902, 3259, 1216, 5543, 2345,  671,  763, 8024,
          2600,  860, 6820,  679, 7231,  102,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   